In [44]:
import tensorflow as tf
from tensorflow import keras
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import random
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import os
import tempfile

In [45]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11972072173442745435
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1734816564
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 4182668153557217124
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

In [46]:
def my_seed_everywhere(seed: int = 42):
    random.seed(seed) # random
    np.random.seed(seed) # np
    os.environ["PYTHONHASHSEED"] = str(seed) # os
    tf.random.set_seed(seed) # tensorflow

my_seed = 42
my_seed_everywhere(my_seed)

In [47]:
train = pd.read_csv("./playground-series-s4e7/train.csv")
target = pd.read_csv("./playground-series-s4e7/test.csv")

In [48]:
neg, pos = np.bincount(train['Response'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Examples:
    Total: 11504798
    Positive: 1415059 (12.30% of total)



In [49]:
category_columns = train.select_dtypes(include="object").columns
category_columns = category_columns.tolist()

def label_encoding(df):
    # Instance of LabelEncoder
    label_encoders = {col: LabelEncoder() for col in category_columns}

    # Label Encoding
    for col in category_columns:
        df[col] = label_encoders[col].fit_transform(df[col])

label_encoding(train)

In [50]:
category_columns = target.select_dtypes(include="object").columns
category_columns = category_columns.tolist()

def label_encoding(df):
    # Instance of LabelEncoder
    label_encoders = {col: LabelEncoder() for col in category_columns}

    # Label Encoding
    for col in category_columns:
        df[col] = label_encoders[col].fit_transform(df[col])

label_encoding(target)

In [51]:
train = train.drop(columns=['id'])
target = target.drop(columns=['id'])

In [52]:
target

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,0,20,1,47.0,0,1,0,2630.0,160.0,228
1,1,47,1,28.0,0,0,1,37483.0,124.0,123
2,1,47,1,43.0,0,0,1,2630.0,26.0,271
3,0,22,1,47.0,1,1,0,24502.0,152.0,115
4,1,51,1,19.0,0,0,0,34115.0,124.0,148
...,...,...,...,...,...,...,...,...,...,...
7669861,1,57,1,28.0,0,0,1,51661.0,124.0,109
7669862,1,28,1,50.0,1,1,0,25651.0,152.0,184
7669863,1,47,1,33.0,1,0,0,2630.0,138.0,63
7669864,1,30,1,28.0,0,1,1,38866.0,124.0,119


In [53]:
# Use a utility from sklearn to split and shuffle your dataset.
train_df, test_df = train_test_split(train, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('Response'))
bool_train_labels = train_labels != 0
val_labels = np.array(val_df.pop('Response'))
test_labels = np.array(test_df.pop('Response'))

train_features = np.array(train_df)
val_features = np.array(val_df)
test_features = np.array(test_df)

In [54]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)

val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)
target_features = scaler.transform(target)

train_features = np.clip(train_features, -5, 5)
val_features = np.clip(val_features, -5, 5)
test_features = np.clip(test_features, -5, 5)


print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)
print('Target features shape:', target_features.shape)

C:\Users\OSY\anaconda3\envs\osy\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Training labels shape: (7363070,)
Validation labels shape: (1840768,)
Test labels shape: (2300960,)
Training features shape: (7363070, 10)
Validation features shape: (1840768, 10)
Test features shape: (2300960, 10)
Target features shape: (7669866, 10)


In [55]:
import tensorflow as tf
from tensorflow import keras

# Define the metrics
METRICS = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'),
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
    keras.metrics.AUC(name='prc', curve='PR'),  # precision-recall curve
]

def make_model(metrics=METRICS, output_bias=None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    
    model = keras.Sequential([
        keras.layers.Dense(
            8, activation='relu',
            input_shape=(train_features.shape[-1],)
        ),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(
            1, activation='sigmoid',
            bias_initializer=output_bias
        ),
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics
    )

    return model


In [57]:
# Define the metrics
METRICS = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'),
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
    keras.metrics.AUC(name='prc', curve='PR'),  # precision-recall curve
]
def make_model(metrics=METRICS, output_bias=None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    
    model = keras.Sequential([
    keras.layers.Dense(10, activation='relu', input_shape=(train_features.shape[-1],)),
    
    keras.layers.Dropout(0.3),
    # 두 번째 은닉층: 12개의 뉴런, ReLU 활성화 함수
    keras.layers.Dense(10, activation='relu'),
    
    # 드롭아웃 레이어: 50%의 뉴런을 랜덤하게 비활성화
    keras.layers.Dropout(0.3),
    
    # 세 번째 은닉층: 8개의 뉴런, ReLU 활성화 함수
    keras.layers.Dense(5, activation='relu'),
    
    # 출력층: 1개의 뉴런, 시그모이드 활성화 함수, 바이어스 초기화
    keras.layers.Dense(1, activation='sigmoid', bias_initializer=output_bias),
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics
    )

    return model

In [58]:
EPOCHS = 100
BATCH_SIZE = 1024

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [59]:
model = make_model()
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 10)                110       
                                                                 
 dropout_4 (Dropout)         (None, 10)                0         
                                                                 
 dense_17 (Dense)            (None, 10)                110       
                                                                 
 dropout_5 (Dropout)         (None, 10)                0         
                                                                 
 dense_18 (Dense)            (None, 5)                 55        
                                                                 
 dense_19 (Dense)            (None, 1)                 6         
                                                                 
Total params: 281
Trainable params: 281
Non-trainable 

In [60]:
model.predict(train_features[:10])

1/1 [==============================] - 0s 97ms/step


array([[0.847736  ],
       [0.8911819 ],
       [0.8697873 ],
       [0.85211194],
       [0.87210953],
       [0.8629155 ],
       [0.8649453 ],
       [0.93545765],
       [0.9094263 ],
       [0.8710841 ]], dtype=float32)

In [61]:
results = model.evaluate(train_features, train_labels, batch_size=BATCH_SIZE, verbose=0)
print("Loss: {:0.4f}".format(results[0]))

Loss: 1.9317


In [62]:
initial_bias = np.log([pos/neg])
initial_bias

array([-1.96434774])

In [63]:
model = make_model(output_bias=initial_bias)
model.predict(train_features[:10])

1/1 [==============================] - 0s 100ms/step


array([[0.34234032],
       [0.2572313 ],
       [0.27695364],
       [0.44368958],
       [0.2996594 ],
       [0.24795197],
       [0.38128504],
       [0.3003234 ],
       [0.18064061],
       [0.14544916]], dtype=float32)

In [64]:
results = model.evaluate(train_features, train_labels, batch_size=BATCH_SIZE, verbose=0)
print("Loss: {:0.4f}".format(results[0]))

Loss: 0.4594


In [65]:
initial_weights = os.path.join(tempfile.mkdtemp(), 'initial_weights')
model.save_weights(initial_weights)

In [66]:
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.57
Weight for class 1: 4.07


In [67]:
# with tf.device('/GPU:0'):
#     model = make_model()
#     model.load_weights(initial_weights)
#     baseline_history = model.fit(
#         train_features,
#         train_labels,
#         batch_size=BATCH_SIZE,
#         epochs=EPOCHS,
#         callbacks=[early_stopping],
#         validation_data=(val_features, val_labels))

In [68]:
with tf.device('/GPU:0'):
    weighted_model = make_model()
    weighted_model.load_weights(initial_weights)

    weighted_history = weighted_model.fit(
        train_features,
        train_labels,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        callbacks=[early_stopping],
        validation_data=(val_features, val_labels),
        class_weight=class_weight)

Epoch 1/100
7191/7191 [==============================] - 67s 9ms/step - loss: 0.4614 - tp: 1789718.0000 - fp: 8933821.0000 - tn: 10439033.0000 - fn: 926638.0000 - accuracy: 0.5536 - precision: 0.1669 - recall: 0.6589 - auc: 0.6039 - prc: 0.1403 - val_loss: 0.3448 - val_tp: 179286.0000 - val_fp: 392018.0000 - val_tn: 1222327.0000 - val_fn: 47137.0000 - val_accuracy: 0.7614 - val_precision: 0.3138 - val_recall: 0.7918 - val_auc: 0.8475 - val_prc: 0.3415
Epoch 2/100
7191/7191 [==============================] - 65s 9ms/step - loss: 0.4433 - tp: 835495.0000 - fp: 2234081.0000 - tn: 4223537.0000 - fn: 69957.0000 - accuracy: 0.6871 - precision: 0.2722 - recall: 0.9227 - auc: 0.8388 - prc: 0.3261 - val_loss: 0.3360 - val_tp: 158067.0000 - val_fp: 325724.0000 - val_tn: 1288621.0000 - val_fn: 68356.0000 - val_accuracy: 0.7859 - val_precision: 0.3267 - val_recall: 0.6981 - val_auc: 0.8482 - val_prc: 0.3413
Epoch 3/100
7191/7191 [==============================] - 65s 9ms/step - loss: 0.4404 - tp: 

In [69]:
y_pred_prob = weighted_model.predict(test_features).ravel()
roc_auc = roc_auc_score(test_labels, y_pred_prob)
print(f'ROC AUC: {roc_auc:.2f}')

71905/71905 [==============================] - 110s 2ms/step
ROC AUC: 0.85
